In [1]:
from google.colab import drive
drive.mount("/content/drive")
import csv
import re
import pickle
import pandas as pd

Mounted at /content/drive


In [2]:
# load data
data_path = '/content/drive/Shareddrives/LOPE/產學合作/中華電信/2020-2021/指向情緒案/data/annot_data/annotated_data_bkup/20210412/aspect_tuples_20210412.1.csv'

with open(data_path, 'r', encoding='utf-8') as f:
  rows = csv.reader(f, delimiter = ',')
  headers = next(rows)
  all_data = [ row for row in rows ]
  evaltext_cntxt = [ data[6] for data in all_data if data[3] == 'True'] # filter evaltext whose is_context is True

In [3]:
pip install ckip_transformers

     |████████████████████████████████| 2.3MB 13.8MB/s 
     |████████████████████████████████| 901kB 50.8MB/s 
     |████████████████████████████████| 3.3MB 47.4MB/s 


In [4]:
import ckip_transformers
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger

In [5]:
%%time
ws_driver = CkipWordSegmenter(level=3, device=0)
pos_driver = CkipPosTagger(level=3, device=0)


CPU times: user 17.7 s, sys: 4.54 s, total: 22.2 s
Wall time: 37.5 s


In [6]:
ws = ws_driver(evaltext_cntxt, use_delim=True)
pos = pos_driver(ws, use_delim=False)

Inference: 100%|██████████| 2/2 [00:06<00:00,  3.35s/it]


In [33]:
def pack_ws_pos_sentece(sentence_ws, sentence_pos):
    assert len(sentence_ws) == len(sentence_pos)
    res = []
    for word_ws, word_pos in zip(sentence_ws, sentence_pos):
        if re.match(r'V.*|N.*|A.*', word_pos) == None:
            res.append(f'{word_ws}\u3000')
    return ' '.join(res)

tagged = []
for sentence_ws, sentence_pos in zip(ws, pos):
    tagged.append(pack_ws_pos_sentece(sentence_ws, sentence_pos))

In [34]:
df = pd.DataFrame([evaltext_cntxt, tagged]).T
df.columns = ['context', 'closed_class']
df

,context,closed_class
0,台星的態度就是在等宿主台哥，逸以待勞，準備寄生。,的 就 是 在 ， ， 。
1,中華、遠傳，蓋一座拆一座,、 ，
2,幹⋯⋯⋯ 過兩個月續約我要換中華,幹 ⋯ ⋯ ⋯ 要
3,用過後，全台整體訊號都不錯的還是基地台最多的中華,過 ， 都 的 還 是 的
4,不知道換成中華會不會好一點,不 會不會 一點
...,...,...
430,遠傳超屌 客戶少了中華3百萬以上 頻寬跟中華只差10M Hz 平均來說 遠傳比中華快多了,屌 了 跟 只 M Hz 來 比 了
431,中華+台哥大合體 & 在5G非吃到飽部分比較有創意,+ & 在 5G 非 到 比較
432,中華>遠傳>>>>>台星>>>>>>>>>台哥,> >>>>> >>>>>>>>>
433,Hami pa,Hami pa


In [28]:
import os
os.chdir('/content/drive/Shareddrives/LOPE/產學合作/中華電信/2020-2021/指向情緒案/sentiment-construction-list')

In [35]:
df.to_csv('context_closed_class.csv')